In [1]:
from utils import get_dataframes

dataframes, effect_data = get_dataframes("", True, False)

In [3]:
import pandas as pd
from utils import combined_damage_statistics


def damage_calculation(row: pd.Series):
    damages = row['damage']
    min_dmg, max_dmg, avg_dmg, std_dmg = combined_damage_statistics(damages)
    return min_dmg, max_dmg, avg_dmg, std_dmg


weapons_df = dataframes["weapons"]
weapons_df[["min_dmg", "max_dmg", "avg_dmg", "std_dmg"]] = weapons_df.apply(
    damage_calculation,
    axis=1,
    result_type="expand"
)

# weapons_df.sort_values(by=["weapon_type", "avg_dmg"])[["name", "weapon_type", "avg_dmg"]]
weapons_df

,name,weapon_type,prerequisites,range_,range_far,damage,abilities,min_dmg,max_dmg,avg_dmg,std_dmg
0,Banditen-Wurfbeilpaar,Äxte,"[[15, Str], [13, Dex]]",5,40,"[[1d8+4, slashing], [1d8+6, slashing]]","[Duo-Waffe, Zweihändig]",12.0,26.0,19.0,3.240370
1,Blutdolch der Sehnsüchte,Dolche,"[[15, Dex]]",5,30,"[[1d8, piercing], [2d8, acid]]",[Doppelter Angriff],3.0,24.0,13.5,3.968627
2,Blutstahlritter-Großschwert,Großschwerter,"[[16, Str], [11, Dex]]",10,0,"[[3d12, slashing], [2d4, thunder]]",[Zweihändig],5.0,44.0,24.5,6.184658
3,Bogen des Morgenrots,Kurzbögen,"[[16, Dex]]",55,80,"[[3d8, piercing], [2d10, fire]]","[Strahl des Morgenrots, Kraft des Morgenrots]",5.0,44.0,24.5,5.678908
4,Doppelschussbogen,Kurzbögen,"[[15, Dex]]",60,90,"[[2d8+2, piercing]]","[Zweihändig, Doppelschuss]",4.0,18.0,11.0,3.240370
5,Dornenbogen,Kurzbögen,"[[14, Dex]]",50,80,"[[2d12+4, piercing], [3d4, piercing]]","[Zweihändig, Dornenrankenexplosion]",9.0,40.0,24.5,5.251984
6,Drachenschlächtergroßaxt,Großäxte,"[[16, Str]]",5,0,"[[2d12, slashing], [4d4, lightning]]","[Zweihändig, Drachenblitz]",6.0,40.0,23.0,5.369668
7,Einfacher Dolch,Dolche,"[[10, Dex]]",5,30,"[[1d4, piercing]]",[Doppelter Angriff],1.0,4.0,2.5,1.118034
8,Elfisches Silberjagdmesser,Dolche,"[[13, Dex]]",5,35,"[[1d4, piercing], [1d8, psychic]]","[Lautloser Wanderer, Doppelter Angriff]",2.0,12.0,7.0,2.549510
9,Erzebrechender Zwergenzweihänder,Großschwerter,"[[13, Str], [12, Dex]]",10,0,"[[2d10, slashing], [2d6, bludgeoning]]",[Zweihändig],4.0,32.0,18.0,4.725816


In [3]:
weapon_abilities_df = dataframes["weapon_abilities"]
weapon_abilities_df

,name,description
0,Deckenbrecher,"Gegen unbelebte Objekte wie Türen, Kisten oder..."
1,Doppelschuss,Mit dieser Waffe können zwei Geschosse gleichz...
2,Doppelter Angriff,Mit der betreffenden Waffe darf doppelt angegr...
3,Dornenrankenexplosion,
4,Drachenblitz,
5,Duo-Waffe,Diese Waffe besteht aus zwei Teilen. Mit diese...
6,Herbstwinde,"Ein rotierender Angriff pro Rast, der Ziele 5f..."
7,Kraft des Morgenrots,Unter Einstrahlung von Tageslicht richtet der ...
8,Lautloser Wanderer,
9,Paralyse,Bei einem Treffer mit der Waffe wird ein d100 ...


In [15]:
def find_strongest_weapon(weapons_df: pd.DataFrame, user_attributes: list[tuple[str, int]],
                          prio_weapon_type: list[str]):
    user_attr_dict = {attr[0].lower(): attr[1] for attr in user_attributes}

    def meets_prerequisites(row):
        prerequisites = row['prerequisites']
        for value, attribute in prerequisites:
            if user_attr_dict.get(attribute.lower(), 0) < int(value):
                return False
        return True

    qualified_weapons = weapons_df[weapons_df.apply(meets_prerequisites, axis=1)]

    if prio_weapon_type:
        prio_weapon_type = [wtype.strip().lower() for wtype in prio_weapon_type]
        qualified_weapons = qualified_weapons.copy()  # Avoid chained assignment issues
        qualified_weapons.loc[:, 'weapon_type'] = qualified_weapons['weapon_type'].str.strip().str.lower()
        qualified_weapons = qualified_weapons[qualified_weapons['weapon_type'].isin(prio_weapon_type)]

    if not qualified_weapons.empty:
        best_weapon = qualified_weapons.loc[qualified_weapons['avg_dmg'].idxmax()]
        return best_weapon[['name', 'weapon_type', 'avg_dmg']].to_dict()
    else:
        return "No weapons match your qualifications."


user_stats = [["str", 14], ["dex", 17]]
prio_weapon_types = ["Kurzbögen"]
best_weapon = find_strongest_weapon(weapons_df, user_stats, prio_weapon_types)
best_weapon


{'name': 'Bogen des Morgenrots', 'weapon_type': 'kurzbögen', 'avg_dmg': 24.5}